## MonteCarlo object & Movers

@Author: 吴炜坤

@email：weikun.wu@xtalpi.com/weikunwu@163.com

在上一个章节中，我们已经对Metropolis算法有了基本的了解。在本章节中，我们将介绍PyRosetta中的MonteCarlo对象以及介绍一些简单movers的使用。在最后我们将示例如何定义一个用于采样的方法流程。

### 一、MonteCarlo object

Metropolis算法有两个关键步骤，那就是move和accept，举例之前在mcmc.py中的代码函数, 判断接受过程中包括三个步骤, 记录move前的构象以及其能量, 采样后判断$P_{accept}$是否接受新的构象。

在PyRosetta里，开发者们已经定义好了一个MonteCarlo的类，这个类主要有两个作用:
- 记录最低低能量的Pose构象;
- 判断是否接受新的构象.

通过直接调用这个类和特定的moves组合使用，就可以实现Metropolis算法，以下我们将举一个实际的例子:

1 首先进行初始化和pose的生成:

In [3]:
# 可视化:
import nglview

# pyrosetta初始化
from pyrosetta import init, pose_from_sequence, create_score_function
from pyrosetta.rosetta.protocols.moves import MonteCarlo
init()
scorefxn = create_score_function('ref2015')

# 从序列生成Pose
pose = pose_from_sequence('AAA')

# 定义温度
kT = 1.0

# 定义MonteCarlo object:
mc = MonteCarlo(pose, scorefxn, kT)

print(f'Old Pose Score:{scorefxn(pose)}')

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release 2020.50+release.1295438cd4bd2be39c9dbbfab8db669ab62415ab 2020-12-12T00:30:01] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python36.Release r274 2020.50+release.1295438cd4b 1295438cd4bd2be39c9dbbfab8db669ab62415ab http://www.pyrosetta.org 2020-12-12T00:30:01
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.6/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=509525957 seed_offset=0 real_seed=509525957 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=509525957 RG_type=mt19937
Old Pose Score:5.50846776888

In [4]:
# old pose
nglview.show_rosetta(pose)

NGLWidget()

2 尝试进行构象的move和接受判断:（重新运行代码之后，结构可能就已经变了）

In [5]:
# 对骨架二面角进行随机的扰动.
import random
new_phi = random.uniform(-180, 180)
new_psi = random.uniform(-180, 180)
pose.set_phi(2, new_phi)
pose.set_psi(2, new_psi)
print(f'New Pose Score:{scorefxn(pose)}')

New Pose Score:21.67053276514237


In [6]:
# 可视化:
nglview.show_rosetta(pose)

NGLWidget()

观察上述的两个构象, 接下来尝试使用MonteCarlo对象进行接受判断:

In [7]:
# accept?
mc.boltzmann(pose)

False

使用mc对象中的函数，可以给出更加详细的接受信息，如下，本次是第n次trials，接受的结果是？

In [8]:
# mc details;
mc.show_scores()
mc.show_counters()

protocols.moves.MonteCarlo: {0} MonteCarlo:: last_accepted_score,lowest_score: 5.50847 5.50847
protocols.moves.TrialCounter: {0}              unk trials=      1 NO ACCEPTS.


#### 小练习

请结合上述的函数，写一个随机采样骨架二面角的MonteCarlo采样程序(100次循环)。

### 二、Movers

> A Mover is a type of object in Rosetta that interacts with a Pose. Frequently, a Mover changes the Pose. 

从Mover的定义直观理解，所有能造成Pose中构象变化的操作，都可成为mover。在上述的例子中，我们通过单一地改变某一个二面角的操作，是蒙特卡洛的一次Move。
接下来将会介绍几个与骨架优化相关的Mover，并且在下面的例子中读者将会学到如何将mover进行组装。

#### 1 Small and Shear Moves

SmallMover和ShearMover是封装好的随机干扰phi/psi二面角的Mover，但有少许不同:
- SmallMover: 随机选择一个氨基酸位点，扰动phi/psi二面角
- ShearMover: 随机选择一个氨基酸位点，扰动phi角，随后将这个数值乘以-1，去干扰psi角。

SmallMover和ShearMover会同时进行骨架的合理性，确保干扰的残基位于Ramachandran plot允许的区域:

<center><img src="./img/rama.jpg" width = "600" height = "200" align=center /></center>

这两个简单mover最常被用于小幅度扰动当前骨架结构的工具。

In [9]:
from pyrosetta.rosetta.protocols.simple_moves import ShearMover, SmallMover
from pyrosetta.rosetta.core.kinematics import MoveMap
movemap = MoveMap()
movemap.set_bb(True)
n_moves = 5  # 定义执行多少次随机扰动
kT = 2.0
small_mover = SmallMover(movemap, kT, n_moves)
shear_mover = ShearMover(movemap, kT, n_moves)

In [10]:
# small_mover/shear_mover 可以设定特定二级结构的扰动范围:
small_mover.angle_max("H", 25)
small_mover.angle_max("E", 25)
small_mover.angle_max("L", 25)

In [11]:
shear_mover.apply(pose)
# 可视化:
nglview.show_rosetta(pose)

core.scoring.ramachandran: {0} shapovalov_lib::shap_rama_smooth_level of 4( aka highest_smooth ) got activated.
basic.io.database: {0} Database file opened: scoring/score_functions/rama/shapovalov/kappa25/all.ramaProb
basic.io.database: {0} Database file opened: scoring/score_functions/rama/flat/avg_L_rama.dat
core.scoring.ramachandran: {0} Reading custom Ramachandran table from scoring/score_functions/rama/flat/avg_L_rama.dat.
basic.io.database: {0} Database file opened: scoring/score_functions/rama/flat/sym_all_rama.dat
core.scoring.ramachandran: {0} Reading custom Ramachandran table from scoring/score_functions/rama/flat/sym_all_rama.dat.
basic.io.database: {0} Database file opened: scoring/score_functions/rama/flat/sym_G_rama.dat
core.scoring.ramachandran: {0} Reading custom Ramachandran table from scoring/score_functions/rama/flat/sym_G_rama.dat.
basic.io.database: {0} Database file opened: scoring/score_functions/rama/flat/sym_P_rama.dat
core.scoring.ramachandran: {0} Reading cus

NGLWidget()

In [12]:
print(f'New Pose Score:{scorefxn(pose)}')

New Pose Score:4.315771159241792


与之前的结果相比，可见使用SmallMover比我们随机在[-180, 180]的区间内随机选择一个数，构象的能量都要更低，构象也更加地合理，因为其检查了打分函数中的Rama项（骨架二面角势）

除了SmallMover和ShearMover以外，Rosetta的基础型Mover类型有非常的多，具体可参考官网链接: https://new.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/Movers-RosettaScripts

#### 2. MinMover


MinMover是Rosetta中被大量使用的一个Mover，基本的方法是使用梯度下降法优化首先对Pose中每一个自由度做偏导计算梯度∇E, 然后以一定的步长去改变当前Pose的各个自由度分量，重新计算当前构象的能量。不断地迭代重复，直到能量收敛到忍受值范围之内。

举一个实际的例子说明: 当前体系中只有4个原子组成的二面角。通过4个原子的坐标可计算第1和第4个原子之间的范德华力得分。通过对两个原子距离的LJ势函数求导，可知两个原子之间的变化方向对能量的影响，选择梯度下降的方向对距离做一个很小的加量$\delta d$, 此时两个原子之间的距离为$d+\delta d$。如此经过几轮迭代，当梯度∇E趋向于0且小于忍受值，迭代停止，构象达到能量较低的状态。

<center><img src="./img/GD.jpg" width = "700" height = "200" align=center /><center>

MinMover是Rosetta中被大量使用的一个Mover通常与Metropolis Monte Carlo连用。MinMover有几个关键的设置，其使用的能量函数、定义Pose自由度的Movemap、梯度下降的方法以及能量的耐受值。

In [13]:
# 初始化定义MinMover
from pyrosetta.rosetta.protocols.minimization_packing import MinMover
# 定义movemap
my_movemap = MoveMap()
my_movemap.set_bb(True)

# 初始化minmover
min_mover = MinMover()
min_mover.movemap(my_movemap)
min_mover.min_type('lbfgs_armijo_nonmonotone')
min_mover.score_function(scorefxn)
min_mover.tolerance(0.01) # 能量变化的耐受值，当小于该值时停止优化.

关于min_type有比较多的选择: 可参考: https://new.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/minimization-overview

此处做出摘要和开发者的评论:
"dfpmin" uses an exact line search, and Jim says it requires ~10 function evaluations per line search.

"dfpmin_armijo" uses an inexact line search, where the step along the search direction only needs to improve the energy by a certain amount, and also make the gradient a certain amount flatter (but not necessarily reach the minimum). This ends up being significantly more efficient, as once it gets going only 2-3 function evaluations are needed per line search, and approximately the same number of line searches are needed as for the exact dfpmin.

"dfpmin_armijo_nonmonotone" uses an even less exact line search along the descent direction, so that the step need only be better than one of the last few points visited. This allows temporary increases in energy, so that the search may escape shallow local minima and flat basins. Jim estimates this is several times more efficient than the exact dfpmin.

"dfpmin_strong_wolfe" uses the More-Thuente line minimization algorithm that enforces both the Armijo and Wolfe conditions. This gives a better parabolic approximation to a minimum and can run a little faster than armijo.

"lbfgs_armijo" uses the L-BFGS minimizer implementation with the Armijo inexact line search conditions.

"lbfgs_armijo_nonmonotone" uses the L-BFGS minimizer implementation with the even more inexact line search conditions.

"lbfgs_strong_wolfe" uses the L-BFGS minimizer implementation with the Wolfe conditions.

In [14]:
# 读入预先生成的高能量构象:
from pyrosetta.io import pose_from_pdb
bad_pose = pose_from_pdb('./data/HighEnergy.pdb')

core.import_pose.import_pose: {0} File './data/HighEnergy.pdb' automatically determined to be of type PDB


In [15]:
# 设置在Pymol中进行能量最小化的观察;
from pyrosetta.teaching import PyMOLMover
from pyrosetta.rosetta.protocols.moves import AddPyMOLObserver_to_conformation
pmm = PyMOLMover()
pmm.keep_history(True)
pmm.apply(bad_pose)
AddPyMOLObserver_to_conformation(bad_pose, True)

In [16]:
# 进行能量最小化
min_mover.apply(bad_pose)

<center><img src="./img/minmover.gif" width = "400" height = "200" align=center /><center>

观察可得能量最小化的轨迹。

#### 3. Combination Mover

在Rosetta中有封装好的一些Combination Mover可以控制流程的运行逻辑，使用他们可以很方便的构建起一个MC流程。以下将着重介绍:

In [17]:
# 初始化
from pyrosetta.rosetta.protocols.moves import SequenceMover, RandomMover

**SequenceMover和RandomMover**都是可以通过add_mover来设置子Mover的。他们的控制逻辑有些不同:
- SequenceMover: 按顺序将子Movers逐一执行;
- RandomMover: 随机挑选一个子Movers执行;

In [18]:
# SequenceMover
seq_mover = SequenceMover()
seq_mover.add_mover(small_mover)
seq_mover.add_mover(shear_mover)
seq_mover.add_mover(min_mover)
seq_mover.apply(bad_pose)

In [19]:
# RandomMover
rand_mover = RandomMover()
rand_mover.add_mover(small_mover)
rand_mover.add_mover(shear_mover)
rand_mover.add_mover(min_mover)
rand_mover.apply(bad_pose)

<br>

**TrialMover**是将一个Mover执行后与再MonteCarlo对象连用，判断构象是否被接受，它可以以SequenceMover或RandomMover构建的底层Mover的作为输入。同时也可以被SequenceMover或RandomMover所创建的更高层级Mover所包含。

In [20]:
# demo for TrialMover(底层)
from pyrosetta.rosetta.protocols.moves import TrialMover

# 定义打分函数:
scorefxn = create_score_function('ref2015')
# 定义温度
kT = 1.0
# 定义MonteCarlo object:
mc = MonteCarlo(pose, scorefxn, kT)

for i in range(5):
    trial_mover = TrialMover(small_mover, mc)
    trial_mover.apply(pose)
    mc.show_state()

protocols.moves.MonteCarlo: {0} MC: 1  4.31577  4.31577  4.31577  4.31577  0  0  0  rejected
protocols.moves.TrialCounter: {0}            Small trials=      1 NO ACCEPTS.
protocols.moves.MonteCarlo: {0} MC: 1  3.64061  3.64061  3.64061  3.64061  0  0  0  accepted score beat low
protocols.moves.TrialCounter: {0}            Small trials=      2;  accepts= 0.5000;  energy_drop/trial=  -0.33758
protocols.moves.MonteCarlo: {0} MC: 1  3.64061  3.64061  3.64061  3.64061  0  0  0  rejected
protocols.moves.TrialCounter: {0}            Small trials=      3;  accepts= 0.3333;  energy_drop/trial=  -0.22505
protocols.moves.MonteCarlo: {0} MC: 1  3.64061  3.64061  3.64061  3.64061  0  0  0  rejected
protocols.moves.TrialCounter: {0}            Small trials=      4;  accepts= 0.2500;  energy_drop/trial=  -0.16879
protocols.moves.MonteCarlo: {0} MC: 1  4.17161  3.64061  4.17161  3.64061  0  0  0  accepted thermally
protocols.moves.TrialCounter: {0}            Small trials=      5;  accepts= 0.4000;  e

In [21]:
# demo for TrialMover(中层)
rand_mover = RandomMover()
rand_mover.add_mover(trial_mover)
rand_mover.add_mover(min_mover)

In [22]:
# demo for TrialMover(顶层)
top_mc = MonteCarlo(pose, scorefxn, kT)
top_trial_mover = TrialMover(rand_mover, top_mc)
for i in range(5):
    top_trial_mover.apply(pose)
    top_mc.show_state()

protocols.moves.MonteCarlo: {0} MC: 1  4.06398  4.06398  4.06398  4.06398  0  0  0  accepted score beat low
protocols.moves.TrialCounter: {0}         MinMover trials=      1;  accepts= 1.0000;  energy_drop/trial=  -0.10763
protocols.moves.MonteCarlo: {0} MC: 1  4.17161  4.06398  4.17161  4.06398  0  0  0  accepted thermally
protocols.moves.TrialCounter: {0}         MinMover trials=      1;  accepts= 1.0000;  energy_drop/trial=  -0.10763
protocols.moves.TrialCounter: {0}        MoverBase trials=      1;  accepts= 1.0000;  energy_drop/trial=   0.10763
protocols.moves.MonteCarlo: {0} MC: 1  2.93786  2.93786  2.93786  2.93786  0  0  0  accepted score beat low
protocols.moves.TrialCounter: {0}         MinMover trials=      1;  accepts= 1.0000;  energy_drop/trial=  -0.10763
protocols.moves.TrialCounter: {0}        MoverBase trials=      2;  accepts= 1.0000;  energy_drop/trial=  -0.56306
protocols.moves.MonteCarlo: {0} MC: 1  2.822  2.822  2.822  2.822  0  0  0  accepted score beat low
protoc

<br>

上面的代码也可以改写使用**RepeatMover**来写

In [23]:
# demo for RepeatMover(顶层)
from pyrosetta.rosetta.protocols.moves import RepeatMover
top_mc = MonteCarlo(pose, scorefxn, kT)
top_trial_mover = TrialMover(rand_mover, top_mc)
rmover = RepeatMover(top_trial_mover, 5)
rmover.apply(pose)

**值得注意的是，不同层级的TrialMover需要有它单独独享的一个MonteCarlo object。试想下，如果公用一个MonteCarlo object会发生什么？**


综上所述，灵活使用这些Combination Mover可以构建出多种多样的搜索逻辑。

#### 小练习

Use the Mover constructs to create a complex folding algorithm. Create a simple program to do the following:

a. Five small moves

b. Minimize

c. Five shear moves

d. Minimize

e. Monte Carlo Metropolis criterion

f. Repeat a–e 100 times

g. Repeat a–f five times, each time decreasing the magnitude of the small and shear moves from 25° to 5° in 5° increments.

Sketch a flowchart, and submit both the flowchart and your code.